In [4]:
import math

# Distances

In [4]:
#Manhattan
def manhattan_dist(p1: tuple,p2: tuple):
    return (abs(p1[0]-p2[0])+abs(p1[1]-p2[1]))

In [5]:
#Euclidean
def euclidean_dist(p1: tuple,p2: tuple):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

In [6]:
#Chebyshev
def chebyshev_dist(p1: tuple,p2: tuple):
    return max(abs(p1[0]-p2[0]),abs(p1[1]-p2[1]))

# Normalization

In [1]:
#Min-Max-Norm
def min_max_norm(l: list):
    ma = max(l)
    mi = min(l)
    li = [(i - mi)/(ma-mi) for i in l]
    return li

In [5]:
#Z-Norm
from statistics import variance, mean
def z_norm(l: list):
    v = variance(l)
    m = mean(l)
    li = [(i-m)/math.sqrt(v) for i in l]
    return li

# Metrics

In [6]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

#Call these functions with the arguments (ground-truth, prediction)

In [ ]:
import pandas as pd
def entropy(df: pd.DataFrame, class_row):
    ...
    